In [1]:
pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

In [79]:
box = pd.DataFrame()
poss = pd.DataFrame()
for i in range(17, 25):
    box_ = pd.read_csv('data/box/box_{}.csv'.format(i))
    poss_ = pd.read_csv('data/possessions/poss_{}.csv'.format(i))
    box = pd.concat([box, box_])
    poss = pd.concat([poss, poss_])


In [80]:
import pandas as pd
import numpy as np

def assign_off_def_vectorized(df):
    # Create boolean mask: True if home is on offense, False if away is on offense
    mask_home_offense = df["possession_team"] == df["home_team_id"]

    # For each player index from 1 to 5, pick from home/away columns based on the mask
    for i in range(1, 6):
        # Offensive columns
        off_col = f"off{i}"
        home_col = f"home_player_{i}"
        away_col = f"away_player_{i}"
        
        # If mask_home_offense is True, pick the home column, else the away column
        df[off_col] = np.where(mask_home_offense, df[home_col], df[away_col])
        
        # Defensive columns
        def_col = f"def{i}"
        # If home is on offense, away is on defense, else home is on defense
        df[def_col] = np.where(mask_home_offense, df[away_col], df[home_col])

    # Now drop the old home/away columns
    cols_to_drop = [f"home_player_{i}" for i in range(1,6)] + [f"away_player_{i}" for i in range(1,6)]
    df.drop(columns=cols_to_drop, inplace=True)

    return df

# Example usage (assuming df has:
# home_player_1..5, away_player_1..5, possession_team, home_team_id, away_team_id):
poss = assign_off_def_vectorized(poss)


In [81]:
import pandas as pd

# Ensure proper sorting by game_id and game_play_number
poss = poss.sort_values(by=["game_id", "game_play_number"])

# Fill NaN values in possession_team with the last valid value (forward fill)
poss["possession_team_filled"] = poss["possession_team"].fillna(method="ffill")

# Group by game_id to calculate possession changes
poss["possession_change"] = poss.groupby("game_id")["possession_team_filled"].transform(
    lambda x: x != x.shift(1)
)

# Assign a cumulative possession ID within each game
poss["possession_id"] = poss.groupby("game_id")["possession_change"].cumsum()

# Assign home and away possession IDs
poss["home_poss_id"] = poss.apply(
    lambda row: row["possession_id"] if row["possession_team"] == row["home_team_id"] else None, axis=1
)
poss["away_poss_id"] = poss.apply(
    lambda row: row["possession_id"] if row["possession_team"] == row["away_team_id"] else None, axis=1
)

# Fill NA values in possession IDs with forward-fill (carry over the last valid possession ID)
poss["home_poss_id"] = poss.groupby("game_id")["home_poss_id"].transform(lambda x: x.ffill())
poss["away_poss_id"] = poss.groupby("game_id")["away_poss_id"].transform(lambda x: x.ffill())

# Replace remaining NA values with 0 (for losers of jump ball)
poss["home_poss_id"] = poss["home_poss_id"].fillna(0).astype(int)
poss["away_poss_id"] = poss["away_poss_id"].fillna(0).astype(int)

# Ensure pr


/var/folders/p9/tjxg6pk13wd5gkgl1h535fm80000gn/T/ipykernel_64284/2358153134.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  poss["possession_team_filled"] = poss["possession_team"].fillna(method="ffill")


In [18]:
pd.set_option('display.max_columns', None)

0        2017
1        2017
2        2017
3        2017
4        2017
         ... 
84898    2017
84899    2017
84900    2017
84901    2017
84902    2017
Name: season, Length: 84903, dtype: int64

In [33]:
poss

,Unnamed: 0,game_id,id,sequence_number,type_id,type_text,text,away_score,home_score,period_number,period_display_value,clock_display_value,scoring_play,score_value,possession_team,athlete_id_1,athlete_id_2,athlete_id_3,wallclock,shooting_play,coordinate_x_raw,coordinate_y_raw,season,season_type,away_team_id,away_team_name,away_team_mascot,away_team_abbrev,away_team_name_alt,home_team_id,home_team_name,home_team_mascot,home_team_abbrev,home_team_name_alt,home_team_spread,game_spread,home_favorite,game_spread_available,qtr,time,clock_minutes,clock_seconds,half,game_half,lead_qtr,lead_game_half,start_quarter_seconds_remaining,start_half_seconds_remaining,start_game_seconds_remaining,game_play_number,end_quarter_seconds_remaining,end_half_seconds_remaining,end_game_seconds_remaining,period,lag_qtr,lag_game_half,coordinate_x,coordinate_y,game_date,game_date_time,type_abbreviation,possession_change,possession_id,home_poss_id,away_poss_id,off1,def1,off2,def2,off3,def3,off4,def4,off5,def5,possession_team_filled
0,1,400927392,4009273921,1,615,Jumpball,Erika de Souza vs. Kiah Stokes (Brittany Boyd ...,0,0,1,1st Quarter,10:00,False,0,9.0,2590093.0,2.0,2566210.0,2017-05-13T19:10:13Z,False,25,-8.0,2017,2,17,San Antonio,Stars,SA,San Antonio,9,New York,Liberty,NY,New York,2.5,2.5,True,False,1,10:00,10,0,1,1,1.0,1.0,600,1200,2400,1,600,1200,2400,1,NaN,NaN,49.75,0,2017-05-13,2017-05-13 15:00:00,NaN,True,1,1,0,918,2,2590093,1006,851,2566106,2491197,814,2566210,1004,9.0
1,2,400927392,4009273922,2,20441,Driving Hook Shot,Tina Charles misses 5-foot,0,0,1,1st Quarter,9:45,False,0,9.0,918.0,NaN,NaN,2017-05-13T19:10:27Z,True,27,4.0,2017,2,17,San Antonio,Stars,SA,San Antonio,9,New York,Liberty,NY,New York,2.5,2.5,True,False,1,9:45,9,45,1,1,1.0,1.0,585,1185,2385,2,600,1200,2400,1,1.0,1.0,37.75,-2,2017-05-13,2017-05-13 15:00:00,NaN,False,1,1,0,918,2,2590093,1006,851,2566106,2491197,814,2566210,1004,9.0
2,3,400927392,4009273923,3,586,Offensive Rebound,Tina Charles offensive rebound,0,0,1,1st Quarter,9:45,False,0,9.0,918.0,NaN,NaN,2017-05-13T19:10:28Z,False,25,-8.0,2017,2,17,San Antonio,Stars,SA,San Antonio,9,New York,Liberty,NY,New York,2.5,2.5,True,False,1,9:45,9,45,1,1,1.0,1.0,585,1185,2385,3,585,1185,2385,1,1.0,1.0,49.75,0,2017-05-13,2017-05-13 15:00:00,NaN,False,1,1,0,918,2,2590093,1006,851,2566106,2491197,814,2566210,1004,9.0
3,4,400927392,4009273924,4,20424,LayupShot,Tina Charles misses,0,0,1,1st Quarter,9:44,False,0,9.0,918.0,NaN,NaN,2017-05-13T19:10:31Z,True,26,2.0,2017,2,17,San Antonio,Stars,SA,San Antonio,9,New York,Liberty,NY,New York,2.5,2.5,True,False,1,9:44,9,44,1,1,1.0,1.0,584,1184,2384,4,585,1185,2385,1,1.0,1.0,39.75,-1,2017-05-13,2017-05-13 15:00:00,NaN,False,1,1,0,918,2,2590093,1006,851,2566106,2491197,814,2566210,1004,9.0
4,5,400927392,4009273925,5,586,Offensive Rebound,Brittany Boyd offensive rebound,0,0,1,1st Quarter,9:42,False,0,9.0,2566210.0,NaN,NaN,2017-05-13T19:10:32Z,False,25,-8.0,2017,2,17,San Antonio,Stars,SA,San Antonio,9,New York,Liberty,NY,New York,2.5,2.5,True,False,1,9:42,9,42,1,1,1.0,1.0,582,1182,2382,5,584,1184,2384,1,1.0,1.0,49.75,0,2017-05-13,2017-05-13 15:00:00,NaN,False,1,1,0,918,2,2590093,1006,851,2566106,2491197,814,2566210,1004,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84898,84899,400981432,400981432490,490,584,Substitution,Plenette Pierson enters the game for Seimone A...,76,85,4,4th Quarter,0:8.0,False,0,8.0,491.0,695.0,NaN,2017-10-05T02:27:07Z,False,25,-8.0,2017,3,6,Los Angeles,Sparks,LA,Los Angeles,8,Minnesota,Lynx,MIN,Minnesota,2.5,2.5,True,False,4,0:8.0,0,8,2,2,4.0,2.0,8,8,8,400,8,8,8,4,4.0,2.0,49.75,0,2017-10-04,2017-10-04 20:00:00,NaN,False,233,233,232,571,2529047,956,1013,4257500,2529122,545,812,571,1049,8.0
84899,84900,400981432,400981432

In [56]:
# poss = poss.drop(columns=['Unnamed: 0'])
boxes_ids_years = boxes[['game_id', 'season']].drop_duplicates()
poss = poss.merge(boxes_ids_years, on=['game_id'])

In [64]:
boxes_minutes = boxes[['game_id', 'season', 'minutes', 'athlete_id','athlete_display_name']].groupby(['athlete_id','season'], as_index=False).agg({'minutes':'sum'})
boxes_minutes

,athlete_id,season,minutes
0,2,2017,337.0
1,91,2017,952.0
2,491,2017,509.0
3,537,2017,679.0
4,545,2017,700.0
...,...,...,...
159,3888043,2017,386.0
160,3922629,2017,62.0
161,4038352,2017,307.0
162,4038419,2017,20.0


In [82]:
player_cols = [
    'off1', 'off2', 'off3', 'off4', 'off5',
    'def1', 'def2', 'def3', 'def4', 'def5'
]

In [ ]:
def build_player_list(poss):
    players = list(
        set(list(poss['off1'].unique()) + list(poss['off2'].unique()) + 
            list(poss['off3']) + list(poss['off4'].unique()) + 
            list(poss['off5'].unique()) + list(poss['def1'].unique()) + 
            list(poss['def2'].unique()) + list(poss['def3'].unique()) + 
            list(poss['def4'].unique()) + list(poss['def5'].unique())))
    players.sort()
    return players



def map_players(row_in, players):
    p1 = row_in[0]
    p2 = row_in[1]
    p3 = row_in[2]
    p4 = row_in[3]
    p5 = row_in[4]
    p6 = row_in[5]
    p7 = row_in[6]
    p8 = row_in[7]
    p9 = row_in[8]
    p10 = row_in[9]

    rowOut = np.zeros([len(players) * 2])

    rowOut[players.index(p1)] = 1
    rowOut[players.index(p2)] = 1
    rowOut[players.index(p3)] = 1
    rowOut[players.index(p4)] = 1
    rowOut[players.index(p5)] = 1

    rowOut[players.index(p6) + len(players)] = -1
    rowOut[players.index(p7) + len(players)] = -1
    rowOut[players.index(p8) + len(players)] = -1
    rowOut[players.index(p9) + len(players)] = -1
    rowOut[players.index(p10) + len(players)] = -1

    return rowOut

def convert_to_matricies(possessions, name, players):
    # extract only the columns we need

    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[player_cols].to_numpy()
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_players, 1, stints_x_base, players)

    # Convert the column of target values into a numpy matrix
    stint_Y_rows = possessions[name].to_numpy()

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows

# Convert lambda value to alpha needed for ridge CV
def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0

# Convert RidgeCV alpha back into a lambda value
def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples

def calculate_rapm(train_x, train_y, lambdas, name, players):
    # convert our lambdas to alphas
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]

    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True)

    # fit our training data
    model = clf.fit(train_x, train_y,)

    # convert our list of players into a mx1 matrix
    player_arr = np.transpose(np.array(players).reshape(1, len(players)))

    # extract our coefficients into the offensive and defensive parts
    coef_offensive_array = model.coef_[0:len(players)][np.newaxis].T
    coef_defensive_array = model.coef_[len(players):][np.newaxis].T

    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    player_id_with_coef = np.concatenate([player_arr, coef_offensive_array, coef_defensive_array], axis=1)
    # build a dataframe from our matrix
    players_coef = pd.DataFrame(player_id_with_coef)
    intercept = model.intercept_

    # apply new column names
    players_coef.columns = ['playerId', 'O{0}'.format(name),         
    'D{0}'.format(name)]

    # Add the offesnive and defensive components together (we should really be weighing this to the number of offensive and defensive possession played as they are often not equal).
    players_coef[name] = players_coef['O{0}'.format(name)] + players_coef['D{0}'.format(name)]

    # rank the values
    players_coef['{0} Rank'.format(name)] = players_coef[name].rank(ascending=False)
    players_coef['O{0} Rank'.format(name)] = players_coef[         
        'O{0}'.format(name)].rank(ascending=False)
    players_coef['D{0} Rank'.format(name)] = players_coef[
        'D{0}'.format(name)].rank(ascending=False)

    return players_coef, intercept


def calculate_rapm_by_season(possessions, name, lambdas_rapm, current_year=2023, decay_factor=0.9):
    """
    Calculate RAPM for each season separately and return a DataFrame with results for all seasons.
    
    Parameters:
    possessions: DataFrame with columns including 'season', player_cols (off1..def5), and target column `name`.
    name: the target variable (e.g., 'PPP', 'points_scored')
    lambdas_rapm: list of lambdas to try (e.g., [0.01, 0.05, 0.1])
    current_year: int, the reference year from which decay is computed
    decay_factor: float, exponential decay per season (e.g., 0.9 means 10% downweight per year older)
    
    Returns:
    final_df: DataFrame containing playerId, ORAPM, DRAPM, combined RAPM, ranks, and season for all seasons.
    """
    all_season_results = []
    # Identify unique seasons
    seasons = sorted(possessions['season'].unique())
    
    for season in seasons:
        print(f"Processing season: {season}")
        # Filter for one season's data
        season_data = possessions[possessions['season'] == season].copy()
        
        if season_data.empty:
            continue
        
        # Build the player list for this season
        players = build_player_list(season_data)
        
        # Compute weights based on decay
        season_data['years_ago'] = current_year - season_data['season']
        season_data['weight'] = decay_factor ** season_data['years_ago']

        # Convert to matrices
        train_x, train_y = convert_to_matricies(season_data, name, players)
        sample_weights = season_data['weight'].values

        # Convert lambdas to alphas
        alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas_rapm]

        # Custom CV to pick best alpha
        best_alpha = None
        best_score = -np.inf
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for alpha in alphas:
            cv_scores = []
            for train_index, val_index in kf.split(train_x):
                X_train_fold, X_val_fold = train_x[train_index], train_x[val_index]
                y_train_fold, y_val_fold = train_y[train_index], train_y[val_index]
                w_train_fold = sample_weights[train_index]

                model = Ridge(alpha=alpha, fit_intercept=True)
                model.fit(X_train_fold, y_train_fold, sample_weight=w_train_fold)
                score = model.score(X_val_fold, y_val_fold)
                cv_scores.append(score)

            avg_score = np.mean(cv_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_alpha = alpha

        # Fit final model with best alpha
        final_model = Ridge(alpha=best_alpha, fit_intercept=True)
        final_model.fit(train_x, train_y, sample_weight=sample_weights)

        # Extract coefficients
        player_arr = np.array(players).reshape(-1, 1)
        coef_offensive_array = final_model.coef_[:len(players)].reshape(-1, 1)
        coef_defensive_array = final_model.coef_[len(players):].reshape(-1, 1)

        player_id_with_coef = np.concatenate([player_arr, coef_offensive_array, coef_defensive_array], axis=1)
        season_players_coef = pd.DataFrame(player_id_with_coef, columns=['playerId', 'ORAPM', 'DRAPM'])

        season_players_coef[name] = season_players_coef['ORAPM'] + season_players_coef['DRAPM']
        season_players_coef['RAPM Rank'] = season_players_coef[name].rank(ascending=False)
        season_players_coef['ORAPM Rank'] = season_players_coef['ORAPM'].rank(ascending=False)
        season_players_coef['DRAPM Rank'] = season_players_coef['DRAPM'].rank(ascending=False)

        intercept = final_model.intercept_

        # Add season column
        season_players_coef['season'] = season
        season_players_coef['best_alpha'] = best_alpha
        season_players_coef['intercept'] = intercept

        # Append to all season results
        all_season_results.append(season_players_coef)

    # Combine all seasons into one DataFrame
    if all_season_results:
        final_df = pd.concat(all_season_results, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['playerId', 'ORAPM', 'DRAPM', name,
                                         'RAPM Rank', 'ORAPM Rank', 'DRAPM Rank', 'season', 'best_alpha', 'intercept'])

    return final_df



def calculate_rapm_three_year_window(possessions, name, lambdas_rapm, decay_factor=0.9):
    """
    Calculate RAPM for a three-year window for each season and return a DataFrame with results.
    
    Parameters:
    possessions: DataFrame with columns including 'season', player_cols (off1..def5), and target column `name`.
    name: the target variable (e.g., 'PPP', 'points_scored')
    lambdas_rapm: list of lambdas to try (e.g., [0.01, 0.05, 0.1])
    decay_factor: float, exponential decay per season (e.g., 0.9 means 10% downweight per year older)
    
    Returns:
    final_df: DataFrame containing playerId, ORAPM, DRAPM, combined RAPM, ranks, and season for all seasons.
    """
    all_season_results = []
    # Identify unique seasons
    seasons = sorted(possessions['season'].unique())
    
    for season in seasons:
        print(f"Processing RAPM for {season}, using a three-year window.")
        
        # Determine the three-year window
        start_year = max(season - 2, min(seasons))  # Limit to available data
        window_data = possessions[(possessions['season'] >= start_year) & (possessions['season'] <= season)].copy()
        
        if window_data.empty:
            continue

        # Build the player list for this window
        players = build_player_list(window_data)
        
        # Compute weights based on decay
        window_data['years_ago'] = season - window_data['season']
        window_data['weight'] = decay_factor ** window_data['years_ago']

        # Convert to matrices
        train_x, train_y = convert_to_matricies(window_data, name, players)
        sample_weights = window_data['weight'].values

        # Convert lambdas to alphas
        alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas_rapm]

        # Custom CV to pick best alpha
        best_alpha = None
        best_score = -np.inf
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for alpha in alphas:
            cv_scores = []
            for train_index, val_index in kf.split(train_x):
                X_train_fold, X_val_fold = train_x[train_index], train_x[val_index]
                y_train_fold, y_val_fold = train_y[train_index], train_y[val_index]
                w_train_fold = sample_weights[train_index]

                model = Ridge(alpha=alpha, fit_intercept=True)
                model.fit(X_train_fold, y_train_fold, sample_weight=w_train_fold)
                score = model.score(X_val_fold, y_val_fold)
                cv_scores.append(score)

            avg_score = np.mean(cv_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_alpha = alpha

        # Fit final model with best alpha
        final_model = Ridge(alpha=best_alpha, fit_intercept=True)
        final_model.fit(train_x, train_y, sample_weight=sample_weights)

        # Extract coefficients
        player_arr = np.array(players).reshape(-1, 1)
        coef_offensive_array = final_model.coef_[:len(players)].reshape(-1, 1)
        coef_defensive_array = final_model.coef_[len(players):].reshape(-1, 1)

        player_id_with_coef = np.concatenate([player_arr, coef_offensive_array, coef_defensive_array], axis=1)
        season_players_coef = pd.DataFrame(player_id_with_coef, columns=['playerId', 'ORAPM', 'DRAPM'])

        season_players_coef[name] = season_players_coef['ORAPM'] + season_players_coef['DRAPM']
        season_players_coef['RAPM Rank'] = season_players_coef[name].rank(ascending=False)
        season_players_coef['ORAPM Rank'] = season_players_coef['ORAPM'].rank(ascending=False)
        season_players_coef['DRAPM Rank'] = season_players_coef['DRAPM'].rank(ascending=False)

        intercept = final_model.intercept_

        # Add season column
        season_players_coef['season'] = season
        season_players_coef['best_alpha'] = best_alpha
        season_players_coef['intercept'] = intercept

        # Append to all season results
        all_season_results.append(season_players_coef)

    # Combine all seasons into one DataFrame
    if all_season_results:
        final_df = pd.concat(all_season_results, ignore_index=True)
    else:
        final_df = pd.DataFrame(columns=['playerId', 'ORAPM', 'DRAPM', name,
                                         'RAPM Rank', 'ORAPM Rank', 'DRAPM Rank', 'season', 'best_alpha', 'intercept'])
        
    

    return final_df


In [84]:
# Group by game_id and possession_id
df = (
    poss.groupby(["season","game_id", "possession_id"])
    .agg(
        points_scored=("score_value", "sum"),  # Aggregate points scored during the possession
        home_team_id=("home_team_id", "first"),  # Home team ID
        offense_team=("possession_team", "first"),  # Offense team for the possession
        def1=("def1", "first"),  # Defensive players
        def2=("def2", "first"),
        def3=("def3", "first"),
        def4=("def4", "first"),
        def5=("def5", "first"),
        off1=("off1", "first"),  # Offensive players
        off2=("off2", "first"),
        off3=("off3", "first"),
        off4=("off4", "first"),
        off5=("off5", "first"),
    )
    .reset_index()
)

df['home_off'] = df['offense_team'] == df['home_team_id']


In [135]:
poss.to_csv('data/possessions/full_poss.csv')

In [90]:
player_list = build_player_list(poss)

In [91]:
df['PPP'] = df['points_scored'] * 100

In [92]:
train_x, train_y = convert_to_matricies(df, 'PPP', player_list)

In [95]:
lambdas_rapm = [.01, .05, .1]

In [126]:
results = calculate_rapm_three_year_window(df, 'PPP', lambdas_rapm, decay_factor=0.9)

Processing RAPM for 2017, using a three-year window.
Processing RAPM for 2018, using a three-year window.
Processing RAPM for 2019, using a three-year window.
Processing RAPM for 2020, using a three-year window.
Processing RAPM for 2021, using a three-year window.
Processing RAPM for 2022, using a three-year window.
Processing RAPM for 2023, using a three-year window.
Processing RAPM for 2024, using a three-year window.


In [127]:
results

,playerId,ORAPM,DRAPM,PPP,RAPM Rank,ORAPM Rank,DRAPM Rank,season,best_alpha,intercept
0,2.0,-0.215266,0.492144,0.276878,64.0,89.0,49.0,2017,1193.750,55.083118
1,91.0,2.435747,-1.234224,1.201523,41.0,16.0,119.0,2017,1193.750,55.083118
2,491.0,1.219534,0.389146,1.608680,28.0,43.0,53.0,2017,1193.750,55.083118
3,537.0,0.464738,1.211172,1.675910,27.0,62.0,25.0,2017,1193.750,55.083118
4,545.0,-0.205096,0.114658,-0.090438,71.0,88.0,64.0,2017,1193.750,55.083118
...,...,...,...,...,...,...,...,...,...,...
1746,5017726.0,-0.597570,1.848934,1.251364,71.0,144.0,35.0,2024,828.095,72.104719
1747,5105752.0,-2.007611,-2.282882,-4.290494,220.0,196.0,215.0,2024,828.095,72.104719
1748,5106150.0,-1.695033,-0.431280,-2.126313,176.0,189.0,143.0,2024,828.095,72.104719
1749,5130672.0,-0.194002,-3.025179,-3.219181,207.0,119.0,225.0,2024,828.095,72.104719


In [125]:
box[box['athlete_id'] ==3143136]

,Unnamed: 0,game_id,season,season_type,game_date,game_date_time,athlete_id,athlete_display_name,team_id,team_name,team_location,team_short_display_name,minutes,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,fouls,plus_minus,points,starter,ejected,did_not_play,reason,active,athlete_jersey,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_display_name,team_uid,team_slug,team_logo,team_abbreviation,team_color,team_alternate_color,home_away,team_winner,team_score,opponent_team_id,opponent_team_name,opponent_team_location,opponent_team_display_name,opponent_team_abbreviation,opponent_team_logo,opponent_team_color,opponent_team_alternate_color,opponent_team_score
472,473,401018980,2018,2,2018-08-19,2018-08-19 18:00:00,3143136.0,Jaime Nared,17,Aces,Las Vegas,Aces,13.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0,1.0,0.0,0.0,0.0,1.0,-1,2.0,False,False,False,COACH'S DECISION,True,31.0,J. Nared,https://a.espncdn.com/i/headshots/wnba/players...,Guard,G,Las Vegas Aces,s:40~l:59~t:17,las-vegas-aces,https://a.espncdn.com/i/teamlogos/wnba/500/lv.png,LV,000000,c7cfd4,home,False,78,20,Dream,Atlanta,Atlanta Dream,ATL,https://a.espncdn.com/i/teamlogos/wnba/500/atl...,d02030,383a37,93
1498,1499,401105069,2019,2,2019-08-18,2019-08-18 18:00:00,3143136.0,Jaime Nared,17,Aces,Las Vegas,Aces,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,False,False,False,COACH'S DECISION,True,31.0,J. Nared,https://a.espncdn.com/i/headshots/wnba/players...,Guard,G,Las Vegas Aces,s:40~l:59~t:17,las-vegas-aces,https://a.espncdn.com/i/teamlogos/wnba/500/lv.png,LV,000000,c7cfd4,away,True,100,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,85


In [ ]:
results = np.round(results, decimals=2)
results = results.reindex(sorted(results.columns), axis=1)
box = box.drop_duplicates(subset=['athlete_id','season'])
results = results.merge(box[['athlete_id','athlete_display_name','team_name','season']], left_on=['playerId','season'], right_on=['athlete_id','season'], how='left')
results = results.rename(columns={'athlete_display_name': 'Player Name', 'PPP':'RAPM'})


TypeError: DataFrame.dropna() got an unexpected keyword argument 'columns'

In [130]:
results = results.dropna(subset=['Player Name'])

In [132]:
results.sort_values(by='RAPM', ascending=False).head(20)

,DRAPM,DRAPM Rank,ORAPM,ORAPM Rank,RAPM,RAPM Rank,best_alpha,intercept,playerId,season,athlete_id,Player Name,team_name
764,3.37,20.0,8.15,3.0,11.52,1.0,628.18,61.94,3146151.0,2020,3146151.0,Ariel Atkins,Mystics
844,-0.13,114.0,9.75,1.0,9.61,1.0,618.84,64.45,872.0,2021,872.0,Angel McCoughtry,Aces
622,2.51,34.0,6.37,5.0,8.89,2.0,628.18,61.94,872.0,2020,872.0,Angel McCoughtry,Aces
1597,0.56,88.0,8.29,1.0,8.85,1.0,828.10,72.10,3065570.0,2024,3065570.0,Kelsey Plum,Aces
923,1.10,63.0,7.22,4.0,8.32,2.0,618.84,64.45,2998928.0,2021,2998928.0,Breanna Stewart,Storm
966,0.78,79.0,6.99,6.0,7.78,3.0,618.84,64.45,3146151.0,2021,3146151.0,Ariel Atkins,Mystics
986,1.14,61.0,6.53,8.0,7.67,4.0,618.84,64.45,3917450.0,2021,3917450.0,Napheesa Collier,Lynx
882,4.36,7.0,3.17,23.0,7.53,5.0,618.84,64.45,2491877.0,2021,2491877.0,Elena Delle Donne,Mystics
849,-1.01,147.0,8.53,2.0,7.52,6.0,618.84,64.45,981.0,2021,981.0,Courtney Vandersloot,Sky
854,4.05,11.0,3.16,24.0,7.20,7.0,618.84,64.45,1008.0,2021,1008.0,Jasmine Thomas,Sun


In [121]:
results.loc[results['season']==2024].groupby(['team_name'],as_index=False).agg({'RAPM':'sum'}).sort_values(by='RAPM', ascending=False)

,team_name,RAPM
3,Liberty,21.91
10,Sun,13.66
4,Lynx,10.96
9,Storm,9.72
2,Fever,9.03
6,Mystics,8.90
0,Aces,7.59
8,Sparks,2.67
1,Dream,0.71
7,Sky,-2.90
